Copyright Jana Schaich Borg/Attribution-NonCommercial 4.0 International (CC BY-NC 4.0)

# The main reasons to use subqueries are:

+ Sometimes they are the most logical way to retrieve the information you want  
+ They can be used to isolate each logical part of a statement, which can be helpful for troubleshooting long and complicated queries  
+ Sometimes they run faster than joins  

Some people find subqueries easier to read than joins.  However, that is often a result of not feeling comfortable with the concepts behind joins in the first place (I prefer join syntax, so admittedly, that is my preference).

# Subqueries must be enclosed in parentheses.  Subqueries have a couple of rules that joins don't:

+ ORDER BY phrases cannot be used in subqueries (although ORDER BY phrases can still be used in outer queries that contain subqueries).  

+ Subqueries in SELECT or WHERE clauses that return more than one row must be used in combination with operators that are explicitly designed to handle multiple values, such as the IN operator.  Otherwise, subqueries in SELECT or WHERE statements can output no more than 1 row.

# So why would you use subqueries?  

## "On the fly calculations" (or, doing calculations as you need them)

One of the main uses of subqueries is to calculate values as you need them.  This allows you to use a summary calculation in your query without having to enter the value outputted by the calculation explicitly.  

A situation when this capability would be useful is if you wanted to see all the records that were greater than the average value of a subset of your data.  

```sql
SELECT *
FROM exam_answers 
WHERE TIMESTAMPDIFF(minute,start_time,end_time) >
    (SELECT AVG(TIMESTAMPDIFF(minute,start_time,end_time)) AS AvgDuration
     FROM exam_answers
     WHERE TIMESTAMPDIFF(minute,start_time,end_time)>0);
```

* Note: you can write the subquery first, make sure it works, then incorporate it into the query.


This example shows you how subqueries allow you retrieve information **dynamically**, rather than having to hard code in specific numbers or names.  This capability is particularly useful when you need to build the output of your queries into reports or dashboards that are supposed to display real-time information.
    
   
## Testing membership

Subqueries can also be useful for assessing whether groups of rows are members of other groups of rows.  To use them in this capacity, we need to know about and practice the IN, NOT IN, EXISTS, and NOT EXISTS operators.  


EXISTS and NOT EXISTS perform similar functions to IN and NOT IN, but EXISTS and NOT EXISTS can only be used in subqueries. 

Unlike IN/NOT IN statements, EXISTS/NOT EXISTS are logical statements.  Rather than returning raw data, per se, EXISTS/NOT EXISTS statements **return a value of TRUE or FALSE**.  As a practical consequence, EXISTS statements are often written **using an asterisk** after the SELECT clause rather than explicit column names.  

We can use EXISTS and a subquery to compare the users who are in the users table and dogs table, similar to what we practiced previously using joins.  


```sql
SELECT DISTINCT u.user_guid AS uUserID
FROM users u
WHERE EXISTS (SELECT *
              FROM dogs d 
              WHERE u.user_guid =d.user_guid);
```

## Accurate logical representations of desired output and Derived Tables

A third situation in which subqueries can be useful is when they simply represent the logic of what you want better than joins.

We saw an example of this in our last MySQL Exercise.  We wanted a list of each dog a user in the users table owns, with its accompanying breed information whenever possible. 

```sql
SELECT u.user_guid AS uUserID, d.user_guid AS dUserID, d.dog_guid AS dDogID, d.breed
FROM users u LEFT JOIN dogs d
   ON u.user_guid=d.user_guid
```

Once we saw the "exploding rows" phenomenon due to duplicate rows, we wrote a follow-up query in Question 7 to assess how many rows would be outputted per user_id when we left joined the users table on the dogs table:

```sql
SELECT u.user_guid AS uUserID, d.user_guid AS dUserID, count(*) AS numrows
FROM users u LEFT JOIN dogs d
   ON u.user_guid=d.user_guid
GROUP BY u.user_guid
ORDER BY numrows DESC
```

However, it would be much more elegant if we could **simply join the distinct UserIDs in the first place**.  There is no way to do that with join syntax, on its own.  However, you can use subqueries in combination with joins to achieve this goal.  

To complete the join on ONLY distinct UserIDs from the users table, we could write:

```sql 
SELECT DistinctUUsersID.user_guid AS uUserID, d.user_guid AS dUserID, count(*) AS numrows
FROM (SELECT DISTINCT u.user_guid 
      FROM users u) AS DistinctUUsersID 
LEFT JOIN dogs d
   ON DistinctUUsersID.user_guid=d.user_guid
GROUP BY DistinctUUsersID.user_guid
ORDER BY numrows DESC
``` 


**<mark> Queries that include subqueries always run the innermost subquery first, and then run subsequent queries sequentially in order from the innermost query to the outermost query. </mark>**

Therefore, the query we just wrote extracts the distinct user_guids from the users table *first*, and then left joins that reduced subset of user_guids on the dogs table. As mentioned at the beginning of the lesson, since the subquery is in the FROM statement, it actually creates a temporary table, called a derived table, that is then incorporated into the rest of the query.
  
* **We are required to give an alias to any derived table we create in subqueries within FROM statements.**  Otherwise there would be no way for the database to refer to the multiple columns within the temporary results we create. 
   
* We need to use this alias every time we want to execute a function that uses the derived table.
* Relatedly, **aliases used within subqueries can refer to tables outside of the subqueries**.  However, **outer queries cannot refer to aliases created within subqueries unless those aliases are explicitly part of the subquery output**. 


Overall, **subqueries and joins can often be used interchangeably**.  Some people strongly prefer one approach over another, but there is no consensus about which approach is best.  When you are analyzing very large datasets, it's a good idea to test which approach will likely be faster or easier to troubleshoot for your particular application.
